# Overview

This is a python script that addresses the following questions:

- What are the most popular departing and destination stations?
- What are the common origination and arrival area clusters (not just based on Zip Code)? 
- What are the top 10 stations with the least number of bikes by the end of the day at 11 pm over the month of September? 
- What are the top 3 stations with the most number of bikes near each of the stations from question 3)?
- Does bike stations encourage multi-modal travelling?
- How many trips ends near a subway station?

# Hypothesis Testing
the average number of arrival trips is higher compared to the ones that are remote from any subway stations.

# Outputs
- Statistical and Graphical Analysis
- Scenario Simulation

# Key Methods 
- Pairwise Distance (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html)
- Spatial Visualization

---

## Data Ingestion and Feature Engineering

In [221]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
import warnings
import matplotlib.style as style
warnings.filterwarnings('ignore')
style.use('fivethirtyeight')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [222]:
url = "https://s3.amazonaws.com/tripdata/JC-201709-citibike-tripdata.csv.zip"
bike = pd.read_csv(url, compression = "zip")

In [223]:
url = "https://data.cityofnewyork.us/api/views/he7q-3hwy/rows.csv?accessType=DOWNLOAD"
subway = pd.read_csv(url)

In [224]:
bike.shape

(33119, 15)

In [225]:
subway.shape

(1928, 5)

In [226]:
bike.sample(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
7913,298,2017-09-09 07:55:39,2017-09-09 08:00:37,3203,Hamilton Park,40.727596,-74.044247,3273,Manila & 1st,40.721651,-74.042884,29443,Subscriber,1976.0,1
9811,304,2017-09-11 05:36:02,2017-09-11 05:41:06,3207,Oakland Ave,40.737604,-74.052478,3195,Sip Ave,40.730743,-74.063784,29274,Subscriber,1986.0,2
24670,334,2017-09-23 03:42:06,2017-09-23 03:47:40,3196,Riverview Park,40.744319,-74.043991,3212,Christ Hospital,40.734786,-74.050444,29508,Subscriber,1982.0,1
8205,915,2017-09-09 12:20:46,2017-09-09 12:36:02,3269,Brunswick & 6th,40.726012,-74.050389,3270,Jersey & 6th St,40.725289,-74.045572,29589,Subscriber,1986.0,1
13613,457,2017-09-13 21:38:20,2017-09-13 21:45:58,3185,City Hall,40.717733,-74.043845,3209,Brunswick St,40.724176,-74.050656,29460,Subscriber,1975.0,1
5016,641,2017-09-06 17:47:51,2017-09-06 17:58:32,3269,Brunswick & 6th,40.726012,-74.050389,3213,Van Vorst Park,40.718489,-74.047727,29478,Subscriber,1974.0,2
22271,691,2017-09-21 08:26:45,2017-09-21 08:38:16,3209,Brunswick St,40.724176,-74.050656,3199,Newport Pkwy,40.728745,-74.032108,29227,Subscriber,1995.0,2
24719,253,2017-09-23 08:11:10,2017-09-23 08:15:23,3269,Brunswick & 6th,40.726012,-74.050389,3211,Newark Ave,40.721525,-74.046305,26253,Subscriber,1978.0,1
19899,281,2017-09-19 08:48:27,2017-09-19 08:53:08,3267,Morris Canal,40.712419,-74.038526,3183,Exchange Place,40.716247,-74.033459,29512,Subscriber,1987.0,1
29428,631,2017-09-27 17:46:49,2017-09-27 17:57:20,3185,City Hall,40.717733,-74.043845,3190,Garfield Ave Station,40.710467,-74.070039,27999,Subscriber,1989.0,1


In [227]:
bike['starttime'] = pd.to_datetime(bike['starttime'], infer_datetime_format= True)
bike['stoptime'] = pd.to_datetime(bike['stoptime'], infer_datetime_format= True)

In [228]:
bike['day'] = bike['starttime'].dt.day
bike['start_hour'] = bike['starttime'].dt.hour
bike['end_hour'] = bike['stoptime'].dt.hour
bike['DOW'] = bike['starttime'].dt.dayofweek

In [229]:
bike.sample(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,day,start_hour,end_hour,DOW
11693,428,2017-09-12 13:03:41,2017-09-12 13:10:49,3211,Newark Ave,40.721525,-74.046305,3202,Newport PATH,40.727224,-74.033759,29453,Subscriber,1982.0,1,12,13,13,1
31970,304,2017-09-29 18:22:39,2017-09-29 18:27:44,3211,Newark Ave,40.721525,-74.046305,3269,Brunswick & 6th,40.726012,-74.050389,31705,Subscriber,1986.0,2,29,18,18,4
15347,117,2017-09-15 09:05:55,2017-09-15 09:07:52,3270,Jersey & 6th St,40.725289,-74.045572,3211,Newark Ave,40.721525,-74.046305,26240,Subscriber,1987.0,1,15,9,9,4
9286,280,2017-09-10 14:07:17,2017-09-10 14:11:57,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.041050,26303,Subscriber,1971.0,2,10,14,14,6
32268,560,2017-09-30 05:51:28,2017-09-30 06:00:49,3268,Lafayette Park,40.713464,-74.062859,3186,Grove St PATH,40.719586,-74.043117,31675,Subscriber,1984.0,1,30,5,6,5
17819,512,2017-09-17 14:02:33,2017-09-17 14:11:05,3205,JC Medical Center,40.716540,-74.049638,3197,Hs Don't Use,40.719252,-74.034234,26206,Subscriber,1983.0,1,17,14,14,6
17369,416,2017-09-17 06:39:53,2017-09-17 06:46:50,3185,City Hall,40.717733,-74.043845,3214,Essex Light Rail,40.712774,-74.036486,26224,Subscriber,1988.0,1,17,6,6,6
9517,235,2017-09-10 17:08:19,2017-09-10 17:12:15,3276,Marin Light Rail,40.714584,-74.042817,3186,Grove St PATH,40.719586,-74.043117,26291,Subscriber,1986.0,1,10,17,17,6
31913,197,2017-09-29 17:58:16,2017-09-29 18:01:34,3185,City Hall,40.717733,-74.043845,3213,Van Vorst Park,40.718489,-74.047727,31726,Subscriber,1989.0,1,29,17,18,4
13184,673,2017-09-13 17:20:52,2017-09-13 17:32:06,3213,Van Vorst Park,40.718489,-74.047727,3203,Hamilton Park,40.727596,-74.044247,29659,Subscriber,1978.0,1,13,17,17,2


---

## Quantitative Analysis
- Find all unique station and its location details
- Count the number of trips and group by starting and ending station accordingly
- Create a table of bike station, hour, total # of departure, total # of arrival, # of bikes at the dock, and lat/long; assuming there are 20 bikes at the beginning of the day at 12 a.m.

In [230]:
uni_dep_stations = bike[['start station id', 'start station name', 'start station latitude', 'start station longitude']].drop_duplicates()
uni_arv_stations = bike[['end station id', 'end station name', 'end station latitude', 'end station longitude']].drop_duplicates()

In [231]:
print ("number of unique departure stations: {}".format(uni_dep_stations.shape[0]))

number of unique departure stations: 49


In [232]:
print ("number of unique arrival stations: {}".format(uni_arv_stations.shape[0]))

number of unique arrival stations: 69


In [233]:
# Create a dataset with all unique station id, name, and lat/lon
uni_dep_stations.columns = ["id", "name", "lat", "lon"]
uni_arv_stations.columns = ["id", "name", "lat", "lon"]
uni_station = pd.concat([uni_dep_stations, uni_arv_stations], axis = 0).drop_duplicates()
uni_station.head()

,id,name,lat,lon
0,3183,Exchange Place,40.716247,-74.033459
1,3187,Warren St,40.721124,-74.038051
2,3195,Sip Ave,40.730743,-74.063784
3,3272,Jersey & 3rd,40.723332,-74.045953
5,3186,Grove St PATH,40.719586,-74.043117


In [234]:
print ("number of unique stations: {}".format(uni_station.shape[0]))

number of unique stations: 69


### Top 5 Stations with The Most Arrival Trips in Sept. 2017

In [235]:
start_count = bike.groupby("start station id").size().reset_index(name = "departure_cnt").\
                sort_values(by = "departure_cnt", ascending = False)
start_count = start_count.merge(uni_dep_stations, how = "left", left_on = "start station id", right_on = "id")
start_count.head(5)

,start station id,departure_cnt,id,name,lat,lon
0,3186,4079,3186,Grove St PATH,40.719586,-74.043117
1,3203,2415,3203,Hamilton Park,40.727596,-74.044247
2,3183,2146,3183,Exchange Place,40.716247,-74.033459
3,3195,1672,3195,Sip Ave,40.730743,-74.063784
4,3202,1487,3202,Newport PATH,40.727224,-74.033759


### Top 5 Stations with The Most Departure Trip in Sept. 2017

In [236]:
end_count = bike.groupby("end station id").size().reset_index(name = "arrival_cnt").\
                sort_values(by = "arrival_cnt", ascending = False)
end_count = end_count.merge(uni_arv_stations, how = "left", left_on = "end station id", right_on = "id")
end_count.head(5)

,end station id,arrival_cnt,id,name,lat,lon
0,3186,5005,3186,Grove St PATH,40.719586,-74.043117
1,3183,2585,3183,Exchange Place,40.716247,-74.033459
2,3203,2044,3203,Hamilton Park,40.727596,-74.044247
3,3202,1491,3202,Newport PATH,40.727224,-74.033759
4,3195,1421,3195,Sip Ave,40.730743,-74.063784


### The Stations with the Most and Least Bike Balance on Sept 1, 2017

- create dataset of # of arrival by hour
- create dataset of # of departure by hour
- create dataset with net flow by hour
- create dataset with end-of-hour bike balance; assume 20 bikes at 12am

Technique
- long-to-wide table transpose - https://pandas.pydata.org/pandas-docs/stable/reshaping.html

In [237]:
df = bike[(bike['starttime'].dt.day == 1)]

In [238]:
# Create a dataset of hourly departure by station
hourDep = pd.pivot_table(df[['start station id', 'start_hour']], 
                         index = "start station id", columns = "start_hour", 
                         aggfunc = np.size, fill_value= 0).reset_index()
hourDep.columns = ["dep_" + str(i) for i in hourDep.columns]
hourDep.head(5)

,dep_start station id,dep_0,dep_1,dep_2,dep_4,dep_5,dep_6,dep_7,dep_8,dep_9,dep_10,dep_11,dep_12,dep_13,dep_14,dep_15,dep_16,dep_17,dep_18,dep_19,dep_20,dep_21,dep_22,dep_23
0,3183,6,0,0,2,0,4,6,0,0,6,6,8,12,12,20,22,22,10,16,12,2,4,0
1,3184,0,0,0,0,0,0,2,4,2,4,4,4,8,6,4,0,4,6,6,2,0,0,4
2,3185,0,0,0,0,0,0,0,0,10,2,4,4,8,4,2,10,6,4,0,0,0,0,0
3,3186,2,0,0,0,2,0,4,10,6,4,4,6,16,18,18,28,24,42,36,14,8,16,8
4,3187,4,0,0,0,0,2,6,10,6,8,0,6,4,2,4,4,6,6,0,6,4,0,0


In [239]:
hourDep.shape

(44, 24)

In [240]:
# Create a dataset of hourly arrival by station
hourArv = pd.pivot_table(df[['end station id', 'end_hour']], 
                         index = "end station id", columns = "end_hour", 
                         aggfunc = np.size, fill_value= 0).reset_index()
hourArv.columns = ["arv_" + str(i) for i in hourArv.columns]
hourArv.head(5)

,arv_end station id,arv_0,arv_1,arv_2,arv_4,arv_5,arv_6,arv_7,arv_8,arv_9,arv_10,arv_11,arv_12,arv_13,arv_14,arv_15,arv_16,arv_17,arv_18,arv_19,arv_20,arv_21,arv_22,arv_23
0,3183,0,0,0,0,4,4,26,42,42,14,4,8,20,6,8,6,0,8,6,2,0,0,0
1,3184,0,0,0,0,0,6,6,14,6,6,2,4,6,0,0,0,8,2,0,2,0,0,2
2,3185,0,0,0,0,2,2,4,6,6,2,4,6,8,0,4,0,8,2,0,6,0,0,4
3,3186,2,0,0,2,4,24,52,86,60,28,10,10,8,14,14,14,24,12,8,10,12,4,2
4,3187,0,0,0,0,0,0,2,0,6,4,2,4,6,2,6,10,10,8,12,0,0,2,0


In [241]:
hourArv.shape

(50, 24)

In [242]:
# Create a dataset of hourly net flow by stations (Arrival-departure)

hourNet = uni_station.merge(hourDep, how = "left", left_on = "id", right_on = "dep_start station id").\
                      merge(hourArv, how = "left", left_on = "id", right_on = "arv_end station id").fillna(0)
    
for i in range(0, 24):
    try:
        net_col = "net_"+str(i)
        dep_col = "dep_"+str(i)
        arv_col = "arv_"+str(i)
        hourNet[net_col] = hourNet[arv_col] - hourNet[dep_col]
    except (KeyError):
        print("Missing hour: {}".format(i))
        pass

hourNet.sample(10)

Missing hour: 3


,id,name,lat,lon,dep_start station id,dep_0,dep_1,dep_2,dep_4,dep_5,dep_6,dep_7,dep_8,dep_9,dep_10,dep_11,dep_12,dep_13,dep_14,dep_15,dep_16,dep_17,dep_18,dep_19,dep_20,dep_21,dep_22,dep_23,arv_end station id,arv_0,arv_1,arv_2,arv_4,arv_5,arv_6,arv_7,arv_8,arv_9,arv_10,arv_11,arv_12,arv_13,arv_14,arv_15,arv_16,arv_17,arv_18,arv_19,arv_20,arv_21,arv_22,arv_23,net_0,net_1,net_2,net_4,net_5,net_6,net_7,net_8,net_9,net_10,net_11,net_12,net_13,net_14,net_15,net_16,net_17,net_18,net_19,net_20,net_21,net_22,net_23
10,3207,Oakland Ave,40.737604,-74.052478,3207.0,0.0,0.0,0.0,0.0,4.0,2.0,2.0,12.0,2.0,2.0,2.0,0.0,8.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,3207.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,2.0,8.0,2.0,0.0,2.0,4.0,2.0,2.0,6.0,0.0,0.0,0.0,-4.0,-2.0,-2.0,-12.0,-2.0,-2.0,0.0,0.0,-4.0,-2.0,0.0,2.0,6.0,0.0,0.0,2.0,2.0,2.0,2.0
11,3267,Morris Canal,40.712419,-74.038526,3267.0,0.0,0.0,0.0,0.0,2.0,6.0,12.0,28.0,12.0,6.0,10.0,8.0,10.0,4.0,0.0,2.0,6.0,8.0,4.0,2.0,0.0,0.0,0.0,3267.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,2.0,2.0,10.0,16.0,6.0,4.0,10.0,10.0,4.0,6.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-2.0,-6.0,-8.0,-26.0,-12.0,-4.0,-8.0,2.0,6.0,2.0,4.0,8.0,4.0,-4.0,2.0,2.0,2.0,2.0,0.0
13,3193,Lincoln Park,40.724605,-74.078406,3193.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,3193.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,2.0,2.0,-4.0,0.0,-2.0,0.0,0.0,2.0,2.0,2.0,-2.0,-4.0,2.0,0.0,0.0,0.0,0.0
19,3192,Liberty Light Rail,40.711242,-74.055701,3192.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,10.0,6.0,4.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,3192.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,6.0,4.0,2.0,2.0,2.0,2.0,2.0,4.0,0.0,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,-2.0,-6.0,-4.0,-4.0,2.0,4.0,0.0,-2.0,2.0,2.0,2.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0
36,3281,Leonard Gordon Park,40.745910,-74.057271,3281.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,3281.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-2.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,-2.0,-2.0,0.0,-2.0,0.0
31,3277,Communipaw & Berry Lane,40.714358,-74.066611,3277.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3277.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,-2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
53,3436,Greenwich St & Hubert St,40.721319,-74.010065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,3197,Hs Don't Use,40.719252,-74.034234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,3220,5 Corners Library,40.734961,-74.059503,3220.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,3220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-2.0,0.0,2.0,0.0,0.0
45,3188,NJCU,40.710109,-74.085849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
# Calculate end-of-hour bike balance 

starting_bal = 20
hourBal = hourNet

for i in range(1, 24):
    try:
        bal_col = "bal_" + str(i)
        last_bal_col = "bal_" + str(i-1)
        net_col = "net_" + str(i-1)
        if i == 1:
            hourBal[bal_col] = 20 + hourBal[net_col]
        else:
            hourBal[bal_col] = hourBal[last_bal_col] + hourBal[net_col]
    except (KeyError) as ex:
        # use previous balance for missing time slot
        print("Missing net flow at {} (am/pm): use balance at {} (am/pm) for {} (am/pm)".format(i-1, i-2, i))
        hourBal[bal_col] = hourBal[last_bal_col]
        pass

Missing net flow at 3 (am/pm): use balance at 2 (am/pm) for 4 (am/pm)


In [244]:
# Top 5 stations with the most bike at 11pm, Sept 1, 2017
hourBal.sort_values(by = "bal_23", ascending= False).head(5)[["id", "name", "bal_23"]]

,id,name,bal_23
4,3186,Grove St PATH,160.0
0,3183,Exchange Place,50.0
49,3480,WS Don't Use,42.0
27,3202,Newport PATH,32.0
38,3273,Manila & 1st,30.0


In [245]:
# Top 5 stations with the most bike at 11pm, Sept 1, 2017
hourBal.sort_values(by = "bal_23", ascending= True).head(5)[["id", "name", "bal_23"]]

,id,name,bal_23
11,3267,Morris Canal,-16.0
15,3203,Hamilton Park,-8.0
23,3209,Brunswick St,-4.0
16,3269,Brunswick & 6th,2.0
3,3272,Jersey & 3rd,2.0
